In [12]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

from tpot import TPOTClassifier, TPOTRegressor
from datacleaner import autoclean

import seaborn as sns

In [3]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
df = pd.read_csv(url, sep=None, engine='python', names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'])

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [5]:
df.num.value_counts()

0    164
1     55
2     36
3     35
4     13
Name: num, dtype: int64

O dataset esta dividido em 5 categorias em sua variável target (num) que representa o grau de comprometimento do paciente. Distribuição das classes: 164 (54,1%) não comprometido, 55 + 36 + 35 + 13 comprometido (45,9%) com grau de doença 1-4.

Sendo assim, será aplicado um processamento adicional na variável target para ajustar o dataset a uma classificação binária, "doente e não doente".

In [7]:
df['num'] = df.num.apply(lambda x: 0 if x == 0 else 1)
df.num.value_counts()

0    164
1    139
Name: num, dtype: int64

In [15]:
df = autoclean(df)

target = 'num'
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = TPOTClassifier(generations=3, population_size=50, verbosity=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print()
print('Accuracy: %s - Acc Ref: %s' % (accuracy_score(y_test, y_pred), 0.900))
print()
print(classification_report(y_test, y_pred))

Best pipeline: GaussianNB(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False))

Accuracy: 0.8524590163934426 - Acc Ref: 0.9

             precision    recall  f1-score   support

          0       0.81      0.90      0.85        29
          1       0.90      0.81      0.85        32

avg / total       0.86      0.85      0.85        61

